In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing
import dask.dataframe as dd # data processing
import matplotlib.pyplot as plt # data visualisation
import seaborn as sns # data visualisation
import dataprep # data pre-processing
import category_encoders
sns.set()

In [12]:
path = 'Data/train.csv' # training data path
data = pd.read_csv(path) # read data

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
# pd.set_option('display.max_columns', 100)
data.head(10) # display top 10 rows

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,VAR_0010,VAR_0011,VAR_0012,VAR_0013,VAR_0014,VAR_0015,VAR_0016,VAR_0017,VAR_0018,VAR_0019,VAR_0020,VAR_0021,VAR_0022,VAR_0023,VAR_0024,VAR_0025,VAR_0026,VAR_0027,VAR_0028,VAR_0029,VAR_0030,VAR_0031,VAR_0032,VAR_0033,VAR_0034,VAR_0035,VAR_0036,VAR_0037,VAR_0038,VAR_0039,VAR_0040,VAR_0041,VAR_0042,VAR_0043,VAR_0044,VAR_0045,VAR_0046,VAR_0047,VAR_0048,VAR_0049,...,VAR_1886,VAR_1887,VAR_1888,VAR_1889,VAR_1890,VAR_1891,VAR_1892,VAR_1893,VAR_1894,VAR_1895,VAR_1896,VAR_1897,VAR_1898,VAR_1899,VAR_1900,VAR_1901,VAR_1902,VAR_1903,VAR_1904,VAR_1905,VAR_1906,VAR_1907,VAR_1908,VAR_1909,VAR_1910,VAR_1911,VAR_1912,VAR_1913,VAR_1914,VAR_1915,VAR_1916,VAR_1917,VAR_1918,VAR_1919,VAR_1920,VAR_1921,VAR_1922,VAR_1923,VAR_1924,VAR_1925,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,0.0,...,97,97,0,999999997,999999997,999999997,999999997,999999997,999999997,999999997,1,997,997,997,147,198,173,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1.0,0.0,False,False,False,False,False,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,1.0,...,1,5,1,2684,5472,2700,1094,2788,2788,173,1,1,49,49,3,21,12,2,2,0,2,0,0,0,0,0,6546,6546,6125,76,1,1,9996,111,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,0.0,...,97,97,0,999999997,999999997,999999997,999999997,999999997,999999997,999999997,1,1,997,997,45,46,46,1,0,0,3,0,97,0,97,997,999999997,999999997,999999997,999999997,1,997,9996,113,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,0.0,...,0,2,0,1774,999999996,999999996,999999996,999999996,999999996,113,120,30,996,996,15,188,106,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,0.0,...,0,2,0,568,3405,2700,1135,2837,2837,56,30,1,17,17,76,443,230,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,BRANCH,1
5,14,R,4,4,4422,C,0.0,0.0,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,0.0,...,98,98,98,999999998,999999998,999999998,999999998,999999998,999999998,999999998,998,998,998,998,9998,9998,9998,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,9998,9998,0,98,999999998,999999998,9998,0,98,98,998,999999998,998,998,9998,9998,IAPS,0
6,16,H,60,132,40000,C,1.0,1.0,False,False,False,False,False,1.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,False,[],0.0,0.0,0.0,0.0,0.0,...,97,97,0,999999997,999999997,999999997,999999997,999999997,999999997,999999997,1,1,997,997,69,215,139,0,98,98,98,98,98,98,98,998,999999998,999999998,999999998,999999998,998,998,99

In [14]:
# number of rows and columns in dataset
rows = data.shape[0]
columns = data.shape[1]
print("Data has {} rows, {} columns".format(rows, columns))

Data has 145231 rows, 1934 columns


# DATA PREPROCESSING

## Handle Nulls

In [26]:
# number of nulls in dataset
nulls = (data.isna().sum()/rows)*100

# columns with more than 25% nulls
columns_to_remove = []

# columns with nulls
null_columns = []

for i in nulls.index:
    if(nulls[i] >= 25):
        columns_to_remove.append(i)
    elif(nulls[i]>0):
        null_columns.append(i)


In [28]:
# drop columns with more than 25% of nulls
data = data.drop(columns_to_remove, axis = 1)

print("Columns remaining are {}".format(data.shape[1]))

Columns remaining are 1910


In [34]:
# dropping rows with nulls (atmost 0.68% values are null in a null_columns)
data = data.dropna().reset_index(drop = True)

## Clean Data

In [56]:
# categorical variables
cat_vars = []

for i in data.columns:
    if(data[i].dtype == 'object'):
        cat_vars.append(i)

In [57]:
# columns to drop (with same value across the dataset)
to_drop = ["VAR_0236", "VAR_0239", "VAR_0075", "VAR_0204", "VAR_0217"]

for i in cat_vars:
    if len(data[i].value_counts().index) == 1:
        to_drop.append(i)

In [59]:
data = data.drop(to_drop, axis = 1)

## Handle Categorical variables

In [68]:
# encode categorical variable using LeaveOneOutEncoding technique
from category_encoders import LeaveOneOutEncoder

# columns to encode
encode_cols = []

# binary categorical columns
bin_cat_cols = []

for i in data.columns:
#     categorical columns remaining
    if(i in cat_vars and not (i in to_drop)):
        if(len(data[i].value_counts().index)>2):
#             non-binary valued columns
            encode_cols.append(i)
        else:
#             binary valued columns
            bin_cat_cols.append(i)

# intialise encoder
encoder = LeaveOneOutEncoder(cols = encode_cols)

In [70]:
# data prep for encoding
X = data.drop(['target'], axis = 1)
y = data['target']

# fit and encode data
data = encoder.fit_transform(X, y)
data['target'] = y

# IGNORE

In [60]:
for i in data.columns:
    if(data[i].dtype == 'object'):
        print(data[i].value_counts())

R    77175
H    54340
Q      428
Name: VAR_0001, dtype: int64
B    65369
C    49217
N    14762
S     2595
Name: VAR_0005, dtype: int64
22SEP10:00:00:00    1047
23SEP10:00:00:00     843
06DEC11:00:00:00     695
15NOV11:00:00:00     667
22NOV11:00:00:00     653
                    ... 
19OCT05:00:00:00       1
15JUN05:00:00:00       1
03AUG05:00:00:00       1
24MAR06:00:00:00       1
02FEB06:00:00:00       1
Name: VAR_0075, Length: 2350, dtype: int64
CHICAGO               2012
HOUSTON               1224
JACKSONVILLE          1004
SAN ANTONIO            958
INDIANAPOLIS           836
                      ... 
JEFFERSONVLLE            1
HOLYROOD                 1
FAIRMONT CITY            1
PALM BEACH GARDENS       1
PAINT ROCK               1
Name: VAR_0200, Length: 8651, dtype: int64
31JAN14:15:54:00    219
31JAN14:23:59:00    216
31JAN14:18:30:00    200
31JAN14:23:33:00    198
31JAN14:15:13:00    197
                   ... 
30JAN14:17:46:00      8
29JAN14:20:34:00      8
31JAN14:14:48:0

In [69]:
bin_cat_cols

['VAR_0226', 'VAR_0230', 'VAR_0232', 'VAR_0466']